In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt

In [ ]:
# la fonction link_count est utilisée pour calculer les indicateurs. Elle permet de rattacher les date d'une source de données à des dates fin de mois puis de les compter (groupby).
    
def link_count( alim_dataFrame, target_dataFrame, left_keys, event_date, indicator_name, right_keys=None, type_conso=None):
    # alim_dataFrame dataframe à alimenter
    # target_dataFrame: dataframe contenant l'indicateur à récupérer
    # left_keys : clés servant à réaliser la jointure sur le dataframe à alimenter
    # event_date : date de l'
    # event_name: nom de l'indicateur cible 
    # right_key : facultatif, nom des clés de jointure de droite dans le cas où ces dernière diffère de celle de gauche
    
    tr_target = target_dataFrame.copy()
    tr_target["link_date"] = tr_target[event_date].apply(lambda x: x.replace(day=x.days_in_month))
    tr_target["link_date"] = tr_target["link_date"].astype(str)
    tr_target["link_date"] = tr_target["link_date"].apply(lambda x: x[:10])
    
    tr_alim = alim_dataFrame[left_keys]

    if right_keys == None:
        right_keys = left_keys.copy()[:len(left_keys)-1]
        right_keys.append("link_date")    
        result = tr_alim.merge(tr_target[right_keys], right_on = right_keys, left_on = left_keys, how='left')
        
        if type_conso == None:
            result = result.groupby(left_keys)["link_date"].count().reset_index()
        if type_conso == "unique":
            result = result.groupby(left_keys)["link_date"].nunique().reset_index()
            
            print(result)
            
        result[indicator_name] = result["link_date"]
        result.drop(["link_date"], axis=1, inplace=True)
        alim_dataFrame = alim_dataFrame.merge(result, on=left_keys, how='left')     
        
    else:
        pass
            
    return alim_dataFrame



In [ ]:
def sweep_date(dataframe, id_dataset, start_date, end_date):
# La colonne to_delete contient les offres à supprimer, 1=> à supprimer

    dataframe[start_date] = dataframe[start_date].apply(lambda x: x.replace(hour=0, minute=0, second=0))
    dataframe[end_date] = dataframe[end_date].apply(lambda x: x.replace(hour=0, minute=0, second=0))

  # Etape 1 : Filtre croissant sur id_dataset, start_date, end_date

    dataframe = dataframe.sort_values(by=[id_dataset, start_date, end_date], ascending=[True, True, True])
    dataframe["to_delete"]= 0
    dataframe = dataframe.reset_index(drop=True)

  # Etape 2 : Si pour un même id la date de début inférieure ou égale à la date de fin de la ligne précédente et que la date de fin est inférieure ou égale à la date de fin de la ligne précédente
  # Etape 2: on remplace cette date de fin par celle de la ligne précédente
    for i, val in enumerate(dataframe.iterrows()):
        if i!=0:
            if dataframe.at[i, id_dataset] == dataframe.at[i-1, id_dataset]:
                if dataframe.at[i,start_date]<= dataframe.at[i-1,end_date] :
                    if dataframe.at[i,end_date]<= dataframe.at[i-1,end_date]:
                        dataframe.at[i, end_date] = dataframe.at[i-1,end_date]
  
      dataframe.to_csv(r'/dbfs/FileStore/tables/Etape2.csv')
  # Etape 3: On tri par date décroissante le dataset: id_dataset, start_date, end_date
      dataframe = dataframe.sort_values(by=[id_dataset, start_date, end_date], ascending=[True, False, False])
      dataframe = dataframe.reset_index(drop=True)
  
  # Etape 4: Pour un même id, si la date de fin est inférieure à la de debut de la ligne précédente, on ne fait rien
  # Etape 4bisA: dans le cas contraire, si date de début de la ligne précédente -1 jour est inférieure ou égale à la date fin actuelle alors la date de fin actuelle est remplacée par ligne de fin de la ligne précédente
  # Etape 4bisA: un flag to_delete est ajouté pour suppression pour l'avant dernière étape
  # Etape 4bisB: dans le cas où date de début de la ligne précédente -1 jour est supérieur à la date de fin actuelle, on remplace la date de fin actuelle par la date début de la ligne précédente -1 jour

        for i, val in enumerate(dataframe.iterrows()):
            if i!=0:
                if dataframe.at[i, id_dataset] == dataframe.at[i-1, id_dataset]:
                    if dataframe.at[i, end_date] <  dataframe.at[i-1, start_date]:
                        pass
            else:
                if (dataframe.at[i-1, start_date] - np.timedelta64(1,'D')) <= dataframe.at[i, start_date]:            
                    dataframe.at[i, end_date] = dataframe.at[i, start_date]
                if dataframe.at[i, start_date] == dataframe.at[i-1, start_date]:
                    dataframe.at[i, "to_delete"] = 1
            else:
                dataframe.at[i, end_date] = dataframe.at[i-1, start_date] - np.timedelta64(1,'D')#np.timedelta64(1,'D') dt.timedelta(days=1)

      # Etape 5: on supprimer les ligne à supprimer
        dataframe = dataframe.loc[dataframe["to_delete"] != 1]
        dataframe[end_date] = dataframe[end_date].apply(lambda x: x.replace(hour=23, minute=59, second=59))

      #Etape 6 : on supprime la colonne to_delete
        dataframe = dataframe.drop(["to_delete"], axis=1)

        return dataframe



In [ ]:
def find_duplicates(dataframe, id_dataset, start_date, end_date):
    dataframe = dataframe.sort_values(by=[id_dataset, start_date])
    dataframe["superposition"] = 0
    dataframe = dataframe.reset_index(drop=True)
    
    for i, val in enumerate(dataframe.iterrows()):
        if i!=0:
            if dataframe.at[i, id_dataset] == dataframe.at[i-1, id_dataset]:
                if dataframe.at[i,start_date]<= dataframe.at[i-1,end_date]:
                    dataframe.at[i, "superposition"] = 1
                    dataframe.at[i-1,"superposition"] = 1
    result = dataframe.loc[dataframe["superposition"]==1]
    return result


In [ ]:
def fact_event_gen(dataframe, list_column_qual, list_column_date, list_indicator_names=None):
  # dataframe correspond à la source de données à partir la table de fait va être générée
  # list_column_qual
  # list_column_date correspond à l'ensemble des colonnes date qui vont servir à générer la table de fait, la première colonne doit correspondre à la clé
  # list_indicator_names correspond au nom des indicateurs, si elle est vide, le nom par défaut sera le nom des colonnes
  # Indique la position du premier indicateur dans la list
  
    result = pd.DataFrame()
  
    for col in list_column_qual:
        result[col]= ""


    for i, column in enumerate(list_column_date):
    
        prep = pd.DataFrame()
        list_col_prep = list_column_qual + ["Event_Date"]
        list_col_target =list_column_qual + [column]
        prep[list_col_prep] = dataframe[list_col_target]
        prep = prep.loc[prep["Event_Date"].notnull()].reset_index(drop=True)
        prep[list_indicator_names[i]] = 1
        result = pd.concat([result, prep], sort=False)
        result = result.reset_index(drop=True)

    for i, col in enumerate(list_indicator_names):
        result.loc[result[col].isna()==True, col] = 0
        result[col] = result[col].astype('int64')
      
    return result    